# HSV - Hue, Saturation, Value Colorspace
Stough, 202-

- [skimage's color module](https://scikit-image.org/docs/dev/api/skimage.color.html)
- [What is HSV?](https://en.wikipedia.org/wiki/HSL_and_HSV)
- [Why HSV?](https://dsp.stackexchange.com/questions/2687/why-do-we-use-the-hsv-colour-space-so-often-in-vision-and-image-processing)

The HSV and closely related HSL colorspaces are ways of considering 'color' that 
separate the brightness of a color from its tone and purity. This is useful because we
as humans are highly atuned to *brightness changes* in our perception. So if the RGB space speaks
to the mechanism by which we receive a color stimulus--activation of red, green, and 
blue-centered cones--then the HSV speaks more to how we *perceive* a color.

<img src="../dip_figs/rgb_hsv_diagram.jpeg" style="width:500px;height:200px"/> <img src="../dip_figs/hsl_cylinder.png" style="width:250px;height:200px"/>

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors
import skimage.color as color
from ipywidgets import VBox, HBox, FloatSlider

# For importing from alternative directory sources
import sys  
sys.path.insert(0, '../dip_utils')

import matrix_utils
from vis_utils import (vis_rgb_cube,
                       vis_hsv_cube,
                       vis_hists,
                       lab_uniform)

## Viewing the planes of the HSV space.
Here I'll make a little interactive plot on the HSV space for various saturation, thanks to HYRY's [stackoverflow answer](https://stackoverflow.com/questions/10787103/2d-hsv-color-space-in-matplotlib). We first stack together an HSV image, then convert to RGB so that we can display it.

There is a bit of a problem with these demos. The `plt.ioff()` and `plt.ion()` are required to avoid duplication of plots when putting together an interactive demo. `plt.close()` is used to make sure those plots don't regenerate later in the notebook. 

In [ ]:
def changeSat(HSV, newvalue=1.0):
    H, S, V = np.split(HSV, indices_or_sections=3, axis=-1)
    S = newvalue*np.ones_like(V)
    HSV = np.dstack((H,S,V))
    return color.hsv2rgb(HSV)

In [ ]:
plt.ioff()
# plt.clf()

# Set up the initial display
V, H = np.mgrid[0:1:100j, 0:1:360j]
S = np.ones_like(V)
HSV = np.dstack((H,S,V))
RGB = color.hsv2rgb(HSV)


# A slider to set the saturation.
sat_slider = FloatSlider(
    orientation='horizontal',
    value=1.0,
    min=0.00,
    max=1.00,
    step=0.01,
    description='$S_{HSV}$'
)

# Setting up the figure.
fig_args = {'num':1, 'frameon':True}
fig, ax = plt.subplots(1,1, figsize=(6,3), **fig_args)
fig.canvas.header_visible = False

thedisp = ax.imshow(RGB, origin='lower', extent=[0, 360, 0, 1], aspect=150)
plt.xlabel('H') # These don't need to change
plt.ylabel('V')
thetext = ax.set_title('$S_{HSV}=1$')
plt.tight_layout()

def update_image(change):
    global thedisp, thetext, HSV
    
    I_rgb = changeSat(HSV, change.new)
    thedisp.set_array(I_rgb)
    thetext.set_text(f'$S_{{HSV}}$={change.new:.02f}')
    fig.canvas.draw()
    fig.canvas.flush_events()

sat_slider.observe(update_image, names='value')

VBox([sat_slider, fig.canvas])


In [ ]:
# Close the above interactive demo figure, so that it won't appear later.
plt.close(1)

## Viewing an image as RGB and HSV for comparison

In [ ]:
IMAGE = 'mountainSpring.jpg'
# IMAGE = 'peppers.png'
# IMAGE = 'blueridge.jpg'

# Let's construct our own colormaps
cm_KR = mcolors.LinearSegmentedColormap.from_list('kr', [(0,0,0), (1, 0, 0)], 256)
cm_KG = mcolors.LinearSegmentedColormap.from_list('kg', [(0,0,0), (0, 1, 0)], 256)
cm_KB = mcolors.LinearSegmentedColormap.from_list('kb', [(0,0,0), (0, 0, 1)], 256)

In [ ]:
Irgb = plt.imread('../dip_pics/' + IMAGE)

# Image of uniform random distributed color
# Irgb = np.random.random((100,100,3))

In [ ]:
# plt.clf()
plt.ion()

f, ax = plt.subplots(3,3, figsize = (8,6), sharex=True, sharey=True)

Ihsv = color.rgb2hsv(Irgb)

# Row 0: just the full color image
ax[0][1].imshow(Irgb)
ax[0][0].set_axis_off()
ax[0][2].set_axis_off()

# Row 1: r g b channels
for i in range(3):
    ax[1,i].imshow(Irgb[...,i], cmap=[cm_KR, cm_KG, cm_KB][i])
    ax[1,i].set_title(['Red', 'Green', 'Blue'][i])


# Now to display the HSV space.

cmapNameHue = 'myHue'
colorsH = [(.7,0,0), (.5,.5,0), (0,.7,0), (0,.5,.5), (0,0,.7), (.5, 0, .5), (.7, 0, 0)]
cmHue = mcolors.LinearSegmentedColormap.from_list(cmapNameHue, lab_uniform(colorsH), 256)


ax[2,0].imshow(Ihsv[...,0], cmap=cmHue)
ax[2,0].set_title('H')

ax[2,1].imshow(Ihsv[...,1], cmap='gray')
ax[2,1].set_title('S')

ax[2,2].imshow(Ihsv[...,2], cmap='gray')
ax[2,2].set_title('V')

plt.tight_layout()
f.canvas.set_window_title('RGB vs HSV')
plt.show()

In [ ]:
matrix_utils.arr_info(Ihsv)

&nbsp;

## Let's visualize in 3D

Given the HSV graph above, Hue can be seen as P, and Saturation as R in polar coordinates. Value 
is the Z. Easy. So then, how to [plot in polar coords](https://matplotlib.org/3.1.0/gallery/mplot3d/surface3d_radial.html)?

In [ ]:
vis_hsv_cube(Irgb)

&nbsp;

## Manipulating the HSV components of an image...
In the below interactive visualization we can manipulate the hue of an image. Try to complete the application by adding saturation and value sliders to this image manipulation tool.

Thanks XKCD:

<a href="https://xkcd.com/648/"><img src="https://imgs.xkcd.com/comics/fall_foliage.png" style="height:300px"/></a>


In [ ]:
# I like this image more 
Irgb = plt.imread('../dip_pics/blueridge.jpg')
Ihsv = color.rgb2hsv(Irgb)

In [ ]:
def changeHue(newhue):
    global Ihsv
    change = newhue - Ihsv[...,0].mean()
    
    Ic = Ihsv.copy()
    Ic[...,0] = np.fmod(Ic[...,0]+change, 1) # anything > 1 wraps around 0.
    
    return np.clip(Ic, 0, 1)

In [ ]:
plt.ioff()
# plt.clf()

hue_mean = Ihsv[...,0].mean()

hue_slider = FloatSlider(
    orientation='horizontal',
    value=hue_mean,
    min=0.00,
    max=1.00,
    step=0.01,
    description='Hue'
)

fig_args = {'num':' ', 'frameon':True, 'sharex':True, 'sharey':True}
fig, ax = plt.subplots(1,2, figsize=(8,3), **fig_args) 

I_h = changeHue(hue_mean)
I_rgb = color.hsv2rgb(I_h)

# display artists I'll update
adisp = ax[0].imshow(Irgb)
rdisp = ax[1].imshow(I_rgb)

ax[0].set_title(f'Original ({hue_mean:.03f})')
rtext = ax[1].set_title(f'Hue: {hue_mean:.03f}')


def update_image(change):
    global Ihsv, I_h, adisp, rdisp, rtext
    
    I_rgb = color.hsv2rgb(changeHue(change.new)) 
    rdisp.set_array(I_rgb)
    rtext.set_text(f'Hue: {change.new:.03f}') #hue_slider.value
    fig.canvas.draw()
    fig.canvas.flush_events()

hue_slider.observe(update_image, names='value')

plt.tight_layout()

VBox([hue_slider, fig.canvas])